In [1]:
import pandas as pd
import numpy as np

<p> Geralmente quando começamos a fazer uma análise de dados é bom dar uma olhada no head, shape e info. Estudar os tipos de dados, as colunas, o número de observações, se está faltando algum dado etc </p>

In [2]:
stud= pd.read_csv('data.csv', sep=';')
stud.head()
stud.shape
stud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance	                     4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

In [3]:
#verificar a taxa de evasão escolar
stud['Target'].value_counts(1) # esse 1 obtem o percentual

Graduate    0.499322
Dropout     0.321203
Enrolled    0.179476
Name: Target, dtype: float64

<p> Aqui tiramos que a taxa de evasão, o dropout é de 32%. </p><p>Em seguida, foi pedido para que verificassemos se a taxa de inflação para cada tipo de target poderia indicar essa evasão.</p>

In [4]:
#Média de inflação para cada tipo de target
stud.groupby('Target')['Inflation rate'].mean()

Target
Dropout     1.283955
Enrolled    1.211713
Graduate    1.197918
Name: Inflation rate, dtype: float64

<p> A taxa foi baixa e não parece ser significativa, a sugestão então foi que talvez a situação econômica, o estado civil ou se o estudante mora ou não com os pais poderia ter alguma relação com isso. E para fazer essa análise mais ampla, precisaremos aprender pivot_table no Pandas!! </p>

In [5]:
stud.pivot_table(
    index='Target', 
    columns=['Marital status'], 
    values='Inflation rate', 
    aggfunc=np.mean
    )

Marital status,1,2,3,4,5,6
Target,,,,,,
Dropout,1.282601,1.300559,1.40,1.269048,1.354545,0.875
Enrolled,1.220694,1.003846,2.55,1.593750,-0.333333,1.400
Graduate,1.193499,1.216216,-0.80,1.163636,2.181818,-0.300



<p> O método pivot_table no pandas é usado para criar uma tabela dinâmica (pivot table) a partir de um DataFrame, permitindo agregar e reorganizar os dados de forma flexível </p>

<p> <b>Como funciona a pivot_table </b> </p>
<p>A função pivot_table tem alguns parâmetros principais:</p>

<ul>index: Especifica uma ou mais colunas que serão usadas como índice na tabela dinâmica. Os valores dessas colunas formam as linhas da tabela.

columns: Especifica uma ou mais colunas cujos valores serão usados para criar as colunas da tabela dinâmica.

values: Especifica a(s) coluna(s) cujos valores serão agregados de acordo com a função de agregação (como mean, sum, etc.).

aggfunc: Define a função de agregação a ser usada (por exemplo, np.mean para calcular a média, np.sum para somar, etc.). O padrão é mean.</ul>

In [6]:
stud.pivot_table(
    index='Target',
    columns=['Daytime/evening attendance\t', 'Marital status'],
    values= 'Previous qualification (grade)',
    aggfunc=np.mean
)

Daytime/evening attendance	           0                                        \
Marital status                        1           2      3           4      5   
Target                                                                          
Dropout                      128.471774  130.476471    NaN  127.733333  120.0   
Enrolled                     128.863415  133.296000    NaN  135.400000    NaN   
Graduate                     132.313333  131.133784  120.0  138.388235  120.0   

Daytime/evening attendance	                  1                                 \
Marital status                   6           1           2      3           4   
Target                                                                          
Dropout                      122.5  131.537736  129.937838  135.0  130.753333   
Enrolled                     133.1  131.172607  129.177778  154.0  137.412500   
Graduate                     133.1  134.495393  130.505405    NaN  126.593750   

Daytime/evening attendance	                   
Marital status                      5      6  
Target                                        
Dropout                      143.5500  133.1  
Enrolled                     128.7000    NaN  
Graduate                     132.1375    NaN

<p>Formatação condicional</p>


In [7]:
#criando dataframe
df= pd.DataFrame({
    'A': [9, -7, 5],
    'B': [-1,3,-4]
})
df

,A,B
0,9,-1
1,-7,3
2,5,-4


In [8]:
#definindo função cor
def color_positive_negative(val):
    color='green' if val > 0 else 'red'
    return 'color: %s' % color

#aplicando a função de colocaração no Dataframe

style_df= df.style.applymap(color_positive_negative)
display(style_df)

,A,B
0,9,-1
1,-7,3
2,5,-4


<p> Definindo função que destaca o maximo e o minimo para cada coluna </p>


In [9]:
#definindo a função que destaca max e min

def highlight_max_min(s):
    is_max =s ==s.max()
    is_min =s ==s.min()
    return['color: green' if v_max else 'color: red' if v_min else ''for v_max, v_min in zip (is_max, is_min)]
#aplicando a função
style_df= df.style.apply(highlight_max_min)
display(style_df)

,A,B
0,9,-1
1,-7,3
2,5,-4


<p> vamos criar uma nova pivolt table </p>

In [10]:
pivot_df= stud.pivot_table(
    index='Marital status',
    columns= ['Target'],
    values='Previous qualification (grade)',
    aggfunc=np.mean
)

pivot_df

Target,Dropout,Enrolled,Graduate
Marital status,,,
1,131.216639,131.041111,134.381687
2,130.142458,131.157692,130.819595
3,135.000000,154.000000,120.000000
4,129.890476,136.406250,132.669697
5,141.409091,128.700000,128.827273
6,127.800000,133.100000,133.100000


<p> Agora iremos utilizar uma função para marcar os maiores e menores valores de cada coluna, por isso iremos usar o apply invés do applymap </p>

In [11]:
def highlight_max_min(data):
    styles= data.copy()
    for col in data.columns:
        max_val= data[col].max()
        min_val= data[col].min()
        styles[col]=['background-color: lightgreen' if v == max_val else 'background-color: yellow'
                     if v == min_val else '' for v in data[col]
                     ]
    return styles
#aplicando a função
styled_df= pivot_df.style.apply(highlight_max_min, axis=None)
display(styled_df)

Target,Dropout,Enrolled,Graduate
Marital status,,,
1,131.216639,131.041111,134.381687
2,130.142458,131.157692,130.819595
3,135.000000,154.000000,120.000000
4,129.890476,136.406250,132.669697
5,141.409091,128.700000,128.827273
6,127.800000,133.100000,133.100000


<p>Usando funções aggs: função de agregação para colunas:

podemos usar multiplicas funções aggs desde que colocadas entre <b>['sum', 'mean', 'min'] </b></p>

In [12]:
stud.agg(['sum','mean','min'])

C:\Users\Thiag\AppData\Local\Temp\ipykernel_15400\1910189942.py:1: FutureWarning: ['Target'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  stud.agg(['sum','mean','min'])


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
sum,5214.000000,82592.000000,7644.000000,3.918179e+07,3941.000000,20252.000000,586681.300000,8287.000000,86542.000000,98546.000000,...,2397.000000,27571.000000,35672.000000,19624.000000,45258.430117,665.000000,51168.600000,5432.800000,8.710000,DropoutGraduateDropoutGraduateGraduateGraduate...
mean,1.178571,18.669078,1.727848,8.856643e+03,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969,NaN
min,1.000000,1.000000,0.000000,3.300000e+01,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000,Dropout


<p> É bem parecido com o describe, mas permite mais flexibilização, pois permite você trabalhar com funções próprias e escolher quais quer analisar. </p>

In [13]:
#retorna a diferença entre o valor max e min da serie
def amplitude(series):
    return series.max() - series.min()

stud.agg(['sum', 'mean', 'min','max', amplitude])

C:\Users\Thiag\AppData\Local\Temp\ipykernel_15400\3931180316.py:5: FutureWarning: ['Target'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  stud.agg(['sum', 'mean', 'min','max', amplitude])


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
sum,5214.000000,82592.000000,7644.000000,3.918179e+07,3941.000000,20252.000000,586681.300000,8287.000000,86542.000000,98546.000000,...,2397.000000,27571.000000,35672.000000,19624.000000,45258.430117,665.000000,51168.600000,5432.800000,8.710000,DropoutGraduateDropoutGraduateGraduateGraduate...
mean,1.178571,18.669078,1.727848,8.856643e+03,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969,NaN
min,1.000000,1.000000,0.000000,3.300000e+01,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000,Dropout
max,6.000000,57.000000,9.000000,9.991000e+03,1.000000,43.000000,190.000000,109.000000,44.000000,44.000000,...,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000,Graduate
amplitude,5.000000,56.000000,9.000000,9.958000e+03,1.000000,42.000000,95.000000,108.000000,43.000000,43.000000,...,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,8.600000,4.500000,7.570000,NaN


<p><b> Unindo diferentes Datasets: </b></p>
- dificilmente nas empresas todas as informações que precisamos estarão em um único dataset, por isso é imporante sabermos usar multimos datasetes e o SQL.

<p>vamos emular isso criando data set de compras e geolocalização.</p>


In [14]:
import pandas as pd

# DataFrame de Compras
compras = pd.DataFrame({
    'CompraID': [1, 2, 3, 4, 5],
    'ClienteID': [101, 102, 103, 101, 104],
    'Produto': ['Notebook', 'Smartphone', 'Tablet', 'Smartphone', 'Notebook'],
    'Valor': [2500.00, 1500.00, 1200.00, 800.00, 2700.00]
})

print(compras)


   CompraID  ClienteID     Produto   Valor
0         1        101    Notebook  2500.0
1         2        102  Smartphone  1500.0
2         3        103      Tablet  1200.0
3         4        101  Smartphone   800.0
4         5        104    Notebook  2700.0


In [15]:
# DataFrame de Geolocalização
geolocalizacao = pd.DataFrame({
    'ClienteID': [101, 102, 103, 104],
    'Cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Curitiba'],
    'Latitude': [-23.55052, -22.9068, -19.9167, -25.4284],
    'Longitude': [-46.6333, -43.1729, -43.9345, -49.2733]
})

print(geolocalizacao)


   ClienteID          Cidade  Latitude  Longitude
0        101       São Paulo -23.55052   -46.6333
1        102  Rio de Janeiro -22.90680   -43.1729
2        103  Belo Horizonte -19.91670   -43.9345
3        104        Curitiba -25.42840   -49.2733


<p>nesse caso <b>groupby( )[ ]</b> o parenteses indica a coluna que você quer agrupar e o [] o valor que você ira agregar, <b> o sum() mean() </b> depois disso é a função que será aplicado dentro do parametro dado dentre [] </p>

In [16]:
#somando todas as compras por ID de clientes

compras_group= compras.groupby('ClienteID')['Valor'].sum().reset_index()
compras_group

,ClienteID,Valor
0,101,3300.0
1,102,1500.0
2,103,1200.0
3,104,2700.0


- agora iremos fazer o merge para buscar informações de outros datasets.

In [17]:
compras_group_geo=compras_group.merge(geolocalizacao, on='ClienteID', how='left')
compras_group_geo

,ClienteID,Valor,Cidade,Latitude,Longitude
0,101,3300.0,São Paulo,-23.55052,-46.6333
1,102,1500.0,Rio de Janeiro,-22.90680,-43.1729
2,103,1200.0,Belo Horizonte,-19.91670,-43.9345
3,104,2700.0,Curitiba,-25.42840,-49.2733


- as vezes também é necessário empilhar os dados, como nesse caso que teremos dois datasets com dados de compras de junho e julho e outro de agosto e semtembro, para esse caso podemos usar o concat, que irá jogar os dados das tabelas uma sobre a outra, ou uma do lado da outra


In [18]:
import pandas as pd

# DataFrame de Compras de Junho e Julho
compras_jun_jul = pd.DataFrame({
    'CompraID': [6, 7, 8],
    'ClienteID': [105, 106, 107],
    'Produto': ['Notebook', 'Smartphone', 'Tablet'],
    'Valor': [2600.00, 1300.00, 1500.00],
    'Data': ['2024-06-15', '2024-07-01', '2024-07-20']
})

print(compras_jun_jul)


   CompraID  ClienteID     Produto   Valor        Data
0         6        105    Notebook  2600.0  2024-06-15
1         7        106  Smartphone  1300.0  2024-07-01
2         8        107      Tablet  1500.0  2024-07-20


In [19]:
# DataFrame de Compras de Agosto e Setembro
compras_ago_set = pd.DataFrame({
    'CompraID': [9, 10, 11],
    'ClienteID': [108, 109, 110],
    'Produto': ['Notebook', 'Smartphone', 'Tablet'],
    'Valor': [2700.00, 1400.00, 1600.00],
    'Data': ['2024-08-05', '2024-08-20', '2024-09-10']
})

print(compras_ago_set)


   CompraID  ClienteID     Produto   Valor        Data
0         9        108    Notebook  2700.0  2024-08-05
1        10        109  Smartphone  1400.0  2024-08-20
2        11        110      Tablet  1600.0  2024-09-10


In [20]:
df_concat= pd.concat([compras_jun_jul, compras_ago_set])
df_concat

,CompraID,ClienteID,Produto,Valor,Data
0,6,105,Notebook,2600.0,2024-06-15
1,7,106,Smartphone,1300.0,2024-07-01
2,8,107,Tablet,1500.0,2024-07-20
0,9,108,Notebook,2700.0,2024-08-05
1,10,109,Smartphone,1400.0,2024-08-20
2,11,110,Tablet,1600.0,2024-09-10


<p> <b> Outras funções e funcionalidades do Jupyternotebook. </p> </b>

- A função cut permite você criar categorias baseada em intervalos (segmentar e classficar [bins]).


In [21]:
#criando dataframe contendo valores

df= pd.DataFrame(
    {
        'Value': [1,2,3,4,5,6,7]
    }
)

#usando a função cut para criar intervalos

df['bin']= pd.cut(df['Value'], bins=[0,3,5,7], labels=['Low','Medium','High']) #adiciona uma nova coluna
df

,Value,bin
0,1,Low
1,2,Low
2,3,Low
3,4,Medium
4,5,Medium
5,6,High
6,7,High


<p> Os intervalos são de 0-3 low, 3-5 medium, 5-7 high. Se você tivesse dados fora esses intervalos tipo um 8, daria NaN, valor ausente (missing). </p>

- Inves de passar os intervalos, você também pode dizer em quantas partes você gostaria que o campo continuo fosse quebrado.

In [22]:
#criando dataframe contendo valores

df= pd.DataFrame(
    {
        'Value': [1,2,3,4,5,6,7]
    }
)

#Quebrando os intervalos 
df['equals_bins']= pd.cut(df['Value'], bins=3)
df

,Value,equals_bins
0,1,"(0.994, 3.0]"
1,2,"(0.994, 3.0]"
2,3,"(0.994, 3.0]"
3,4,"(3.0, 5.0]"
4,5,"(3.0, 5.0]"
5,6,"(5.0, 7.0]"
6,7,"(5.0, 7.0]"


- Mas também podemos criar rótulos e até dividir os bins em quantis etc..

In [23]:
#criando campo novo sem rótulos para intervalo

df=pd.DataFrame({'value':[1,2,3,4,5]})
df['quantile_bin']= pd.qcut(df['value'], q=2) 
df

,value,quantile_bin
0,1,"(0.999, 3.0]"
1,2,"(0.999, 3.0]"
2,3,"(0.999, 3.0]"
3,4,"(3.0, 5.0]"
4,5,"(3.0, 5.0]"


In [24]:
#criando com rótulo ficaria assim
df=pd.DataFrame({'value':[1,2,3,4,5]})
df['quantile_bin']= pd.qcut(df['value'], q=2, labels=['Low', 'High']) 
df

#só acrescentamos o labels no código

,value,quantile_bin
0,1,Low
1,2,Low
2,3,Low
3,4,High
4,5,High


<p> A principal funcionalidade desse método na verdade se encontra em poder especificar os quantis, isto é, o intervalo de 10% valores mais baixos ou 50% mais altos. É muito útil no dia a dia, pensa por exemplo, você está indo atrás dos 10% dos clientes que mais consomem etc... </p>


In [25]:
#criando com rótulo ficaria assim
df=pd.DataFrame({'value':[1,2,3,4,5]})
df['quantile_bin']= pd.qcut(df['value'], q=[0,0.1,0.5,1])
df

#nesse caso basta fornecer esses quantis no q= []

,value,quantile_bin
0,1,"(0.999, 1.4]"
1,2,"(1.4, 3.0]"
2,3,"(1.4, 3.0]"
3,4,"(3.0, 5.0]"
4,5,"(3.0, 5.0]"


<p> sample: método para obter uma amostra aleatóris de itens de um eixo de um dataframe. </p>

- por exemplo, nesse caso iremos pegar 10 alunos aleatórios do nosso dataset

In [26]:
#amostra de 10 estudantes

stud= pd.read_csv('data.csv', sep=';')
amostra= stud.sample(n=10)
amostra

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
2168,1,18,2,9500,1,1,131.0,1,1,3,...,0,8,9,8,12.80,0,16.2,0.3,-0.92,Graduate
2316,1,39,1,9119,1,19,133.1,1,1,1,...,0,5,9,2,16.00,4,16.2,0.3,-0.92,Dropout
554,1,17,2,9147,1,1,133.1,1,1,1,...,0,5,5,0,0.00,0,11.1,0.6,2.02,Dropout
2905,1,17,1,9773,1,1,136.0,1,3,1,...,0,6,7,6,12.50,0,10.8,1.4,1.74,Graduate
2340,1,39,1,9003,1,1,111.0,1,37,37,...,0,6,0,0,0.00,0,15.5,2.8,-4.06,Dropout
1567,1,1,2,9147,1,1,108.0,1,1,19,...,0,5,6,5,11.40,0,11.1,0.6,2.02,Graduate
521,1,17,1,9130,1,1,130.0,1,3,19,...,0,5,6,5,12.60,0,9.4,-0.8,-3.12,Graduate
2175,1,1,1,171,1,1,156.0,1,19,38,...,0,0,0,0,0.00,0,9.4,-0.8,-3.12,Graduate
3106,1,1,6,9500,1,1,141.0,1,1,38,...,0,8,8,8,12.32,0,9.4,-0.8,-3.12,Graduate
1846,1,17,1,9003,1,1,126.0,1,37,3,...,0,6,10,4,12.25,0,13.9,-0.3,0.79,Dropout


<p> Também podemos um percentual do data frame, tipo 15% </p>

In [27]:
#amostra de 15% dos estudantes

amostra= stud.sample(frac=0.15)
amostra

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
1731,1,1,1,9130,1,1,136.0,1,19,19,...,0,5,11,3,12.000000,2,10.8,1.4,1.74,Dropout
3379,1,1,4,9500,1,1,148.0,1,19,19,...,0,7,7,6,13.900000,0,7.6,2.6,0.32,Graduate
2546,1,1,1,9238,1,1,135.0,1,19,19,...,0,6,6,6,11.833333,0,10.8,1.4,1.74,Graduate
3829,1,1,1,9085,1,1,148.0,1,38,38,...,0,6,9,4,15.500000,0,11.1,0.6,2.02,Dropout
2670,1,17,5,9119,1,1,138.0,1,19,19,...,0,5,11,5,11.285714,0,12.4,0.5,1.79,Enrolled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,1,39,1,9085,1,12,133.1,1,37,37,...,0,6,0,0,0.000000,0,12.4,0.5,1.79,Dropout
1902,1,1,1,9130,1,1,127.0,1,1,1,...,0,6,9,6,14.000000,0,13.9,-0.3,0.79,Graduate
1335,1,17,2,9853,1,1,126.0,1,1,19,...,0,6,8,5,10.400000,0,7.6,2.6,0.32,Enrolled
2441,1,39,1,8014,0,1,100.0,1,37,37,...,1,6,12,3,11.000000,1,7.6,2.6,0.32,Enrolled


- Por padrão, a amostragem é sem reposição. Se quiser uma amostragem com reposição, utilize o parâmetro replace = True:

<p> <b> Método nunique e unique: </b> </p>

<p><b>nunique</b>:usadas para visualizar os elementos únicos de um objeto
<b>unique:</b> elementos unicos em si </p>



In [28]:
#retorna valores unicos
stud.Target.unique()

array(['Dropout', 'Graduate', 'Enrolled'], dtype=object)

In [29]:
#retorna a quantidade de valores únicos
stud.Target.nunique()

3

In [30]:
#METODO 1
#lista de colunas com tipo de dado 'object' (geralmente texto ou string)
categorical_features=df.select_dtypes(include=['object']).columns.tolist()

#Lista de colunas com tipos de dados númericos (int e float)
numerical_features= df.select_dtypes(exclude=['object']).columns.to_list()

In [31]:
#METODO 2
#Supondo que df é o seu Dataframe e que seu limite é 10 valores únicos

unique_counts= stud.nunique()

categorical_features= unique_counts[unique_counts<=10].index.tolist()
numerical_features= unique_counts[unique_counts>10].index.tolist()

<p> Quando você tem muitas colunas em um DataFrame, é importante separar as colunas categóricas (geralmente texto ou poucos valores distintos) das colunas numéricas (com muitos valores diferentes). Existem duas abordagens para isso: </p>

<ul> Método 1: Separação por Tipo de Dado
Aqui, você utiliza os tipos de dados das colunas para fazer a separação:

Colunas do tipo object geralmente contêm texto e podem ser consideradas categóricas.
Colunas de outros tipos, como int ou float, são tratadas como numéricas.

Método 2: Separação por Número de Valores Únicos
Nesse método, você define um limite de valores únicos para decidir se a coluna é categórica ou numérica:

Se uma coluna tem até um certo número de valores únicos (por exemplo, 10), você a considera categórica.
Caso contrário, ela é considerada numérica. </ul>

<p><b>Como usar funções com apply: </b> </p>

- Função aply permite você aplicar funções personalizadas em linhas ou colunas
- axis=0 coluna
- axis=1 linha
- sem nada será aplicada em todo elemento de uma series 

- Ao usar apply() em um DataFrame, você pode especificar o eixo ao longo do qual a função deve ser aplicada. Por padrão, a função será aplicada a cada coluna (axis=0), mas você pode alterar para aplicar a função a cada linha (axis=1).

In [32]:
#dataframe para exemplos
df_exemplo= pd.DataFrame(
    {
        'A':[1,2,3,4,5],
        'B':[10,20,30,40,50],
        'C':[32,27,24,19,1]
    }
)
df_exemplo

,A,B,C
0,1,10,32
1,2,20,27
2,3,30,24
3,4,40,19
4,5,50,1


In [33]:
#obtendo a media dos alunos
df_exemplo.apply(lambda col: col.mean(), axis=1)

0    14.333333
1    16.333333
2    19.000000
3    21.000000
4    18.666667
dtype: float64

In [34]:
#soma das linhas
df_exemplo.apply(lambda row: row.sum(), axis=0)

A     15
B    150
C    103
dtype: int64

<p> <b> Exemplos de mais aplicando da função apply </p> </b>

In [35]:
df= pd.DataFrame({'A':[1,2,3,4,5]})
df['quadrado']= df['A'].apply(lambda x: x**2)
df

,A,quadrado
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25


In [36]:
stud['new_GDP']= stud.GDP.apply(lambda x: x*100)
stud[['GDP','new_GDP']].head()

,GDP,new_GDP
0,1.74,174.0
1,0.79,79.0
2,1.74,174.0
3,-3.12,-312.0
4,0.79,79.0


<p> <b> Tabelas de funções cruzadas: </p> </b>

variaveis categoricas representam diferentes categorias, rotulos ou classificadores, pense tipo genero, masculino, feminino, cor, marca de carro etc...


<p> Uma tabela de contingência é uma tabela que mostra a frequência com que diferentes combinações de categorias ocorrem em um conjunto de dados. Por exemplo, imagine que você tem um conjunto de dados sobre clientes de uma loja online, com as variáveis <b> "sexo" e "faixa etária" </b>. Uma tabela de contingência mostraria quantas mulheres com idade entre 20 e 30 anos fizeram compras, quantas homens com mais de 50 anos, e assim por diante </p>

In [37]:
prod_df= pd.read_csv('products.csv')
prod_df.head()

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,Trek 820 - 2016,9,6,2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,5,6,2016,749.99
2,3,Surly Wednesday Frameset - 2016,8,6,2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,9,6,2016,2899.99
4,5,Heller Shagamaw Frame - 2016,3,6,2016,1320.99


<p> Queremos saber quantas marcas de bikes foram criadas por ano. Em outras palavras, queremos uma tabela com as colunas sendo o nome da marca (brand_id), as linhas sendo o ano e em cada célula a quantidade de registros para cada ano-marca: </p>

In [38]:
pd.crosstab(prod_df['model_year'], prod_df['brand_id'])

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,12,0,1,3,1,0,0,4,5
2017,16,10,0,0,0,0,23,8,28
2018,90,0,2,0,0,3,0,13,96
2019,0,0,0,0,0,0,0,0,6


<p> Podemos ver o total da soma dos produtos vendidos por linhas e colunas adicionando margins=True </p>

In [39]:
pd.crosstab(prod_df['model_year'], prod_df['brand_id'], margins= True)

brand_id,1,2,3,4,5,6,7,8,9,All
model_year,,,,,,,,,,
2016,12,0,1,3,1,0,0,4,5,26
2017,16,10,0,0,0,0,23,8,28,85
2018,90,0,2,0,0,3,0,13,96,204
2019,0,0,0,0,0,0,0,0,6,6
All,118,10,3,3,1,3,23,25,135,321


<p> Se quisermos saber quanto isso representa em termos de porcentagem precisamos utilziar a função apply </p>

In [40]:
#percentual da linha
pd.crosstab(prod_df['model_year'], prod_df['brand_id']).apply(lambda r: r/r.sum(), axis=1)

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,0.461538,0.000000,0.038462,0.115385,0.038462,0.000000,0.000000,0.153846,0.192308
2017,0.188235,0.117647,0.000000,0.000000,0.000000,0.000000,0.270588,0.094118,0.329412
2018,0.441176,0.000000,0.009804,0.000000,0.000000,0.014706,0.000000,0.063725,0.470588
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [41]:
#percentual para coluna
pd.crosstab(prod_df['model_year'], prod_df['brand_id']).apply(lambda r: r/r.sum(), axis=0)

brand_id,1,2,3,4,5,6,7,8,9
model_year,,,,,,,,,
2016,0.101695,0.0,0.333333,1.0,1.0,0.0,0.0,0.16,0.037037
2017,0.135593,1.0,0.000000,0.0,0.0,0.0,1.0,0.32,0.207407
2018,0.762712,0.0,0.666667,0.0,0.0,1.0,0.0,0.52,0.711111
2019,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.044444


- Identificando valores ausentes com isna()
- Identificando valores ausentes com isna()

<p> esse método é usado para identificar valores ausentes em um dataframe ou serires, vejamos como aplicar ele: </p>



In [42]:
df= pd.DataFrame({
        'A':[1,2, None, 4, 5],
        'B':['A', None, 'C', None, 'E']
    
    })
df

,A,B
0,1.0,A
1,2.0,None
2,NaN,C
3,4.0,None
4,5.0,E


In [43]:
#usando para mostrar se possui dados vazios

df.isna()

,A,B
0,False,False
1,False,True
2,True,False
3,False,True
4,False,False


In [44]:
#contando valores ausentes por coluna
df.isna().sum()

A    1
B    2
dtype: int64

In [45]:
#ausentes por coluna (percentual)
df.isna().mean()

A    0.2
B    0.4
dtype: float64

In [46]:
#filtrando linhas com valores ausentes na coluna A
df[df['A'].isna()]

,A,B
2,NaN,C


In [47]:
##filtrando linhas com valores ausentes na coluna B
df[df['B'].isna()]

,A,B
1,2.0,None
3,4.0,None


- Filtrando Dados com query()

- Query basicamente consiste em filtrar dados utilizando uma string de consulta, tipo o sql, que vai retornar caso seja verdadeiro. vejamos por exemplo

In [48]:
prod.query('list_price>5500')

NameError: name 'prod' is not defined

<p> Retornou todos os produtos cujo preço são superiores a R$5000. </p>

<p> Também podemos usar operadores lógicos dentro de Query() </p>

In [ ]:
prod.query('list_price >5000 & model_year==2018')

,product_id,product_name,brand_id,category_id,model_year,list_price
147,148,Trek Domane SL 8 Disc - 2018,9,7,2018,5499.99
148,149,Trek Domane SLR 8 Disc - 2018,9,7,2018,7499.99
153,154,Trek Domane SLR 6 Disc Women's - 2018,9,7,2018,5499.99
154,155,Trek Domane SLR 9 Disc - 2018,9,7,2018,11999.99
155,156,Trek Domane SL Frameset - 2018,9,7,2018,6499.99
156,157,Trek Domane SL Frameset Women's - 2018,9,7,2018,6499.99
168,169,Trek Emonda SLR 8 - 2018,9,7,2018,6499.99
176,177,Trek Domane SLR 6 Disc - 2018,9,7,2018,5499.99


<p> Agora retornou os valores acima de U$5000, mas apenas do ano de 2018. </p>

<p>Também conseguimos referenciar valores externos: </p>


In [ ]:
preco=5000
prod.query('list_price> @preco & model_year== 2018')

,product_id,product_name,brand_id,category_id,model_year,list_price
147,148,Trek Domane SL 8 Disc - 2018,9,7,2018,5499.99
148,149,Trek Domane SLR 8 Disc - 2018,9,7,2018,7499.99
153,154,Trek Domane SLR 6 Disc Women's - 2018,9,7,2018,5499.99
154,155,Trek Domane SLR 9 Disc - 2018,9,7,2018,11999.99
155,156,Trek Domane SL Frameset - 2018,9,7,2018,6499.99
156,157,Trek Domane SL Frameset Women's - 2018,9,7,2018,6499.99
168,169,Trek Emonda SLR 8 - 2018,9,7,2018,6499.99
176,177,Trek Domane SLR 6 Disc - 2018,9,7,2018,5499.99
